In this notebook we're going to take a close look at the model. We'll start off by defining some constant that we'll use throughout the notebook.

In [1]:
CHECKPOINT_PATH = "./checkpoints/checkpoint.pt"

Then we'll load the checkpoint and instantiate the UltraZoom upscaler.

In [2]:
import torch

from src.ultrazoom.model import UltraZoom

checkpoint = torch.load(
    CHECKPOINT_PATH,
    map_location="cpu",
    weights_only=True,
)

model = UltraZoom(**checkpoint["model_args"])

model.add_weight_norms()

state_dict = checkpoint["model"]

# Compensate for compiled state dict.
for key in list(state_dict.keys()):
    state_dict[key.replace("_orig_mod.", "")] = state_dict.pop(key)

model.load_state_dict(state_dict)

model.remove_parameterizations()

print("Model loaded successfully")

Model loaded successfully


The first module I want to analyze is the Adaptive Residual Mix. I want to see if the model is utilizing its newfound mixing abilities. Let's start by looking at the values of the `alpha` scalar to see if the modules are learning to mix the input and residuals or if it's opting for the identity function.

In [9]:
from src.ultrazoom.model import AdaptiveResidualMix

alphas = []

print("Adaptive Residual Mix alphas:")

for i, module in enumerate(model.modules()):
    if isinstance(module, AdaptiveResidualMix):
        alphas.append(module.alpha.item())

        print(f"Module {i}: alpha = {module.alpha.item():.5f}")

Adaptive Residual Mix alphas:
Module 12: alpha = 0.43228
Module 20: alpha = 0.40428
Module 29: alpha = 0.63633
Module 37: alpha = 0.33727
Module 46: alpha = 0.26273
Module 54: alpha = 0.25766
Module 63: alpha = 0.28875
Module 71: alpha = 0.28878
Module 79: alpha = 0.28840
Module 87: alpha = 0.28869
Module 103: alpha = 0.28978
Module 111: alpha = 0.29050
Module 119: alpha = 0.28895
Module 127: alpha = 0.28842
Module 136: alpha = 0.32022
Module 144: alpha = 0.44893
Module 153: alpha = 0.76267
Module 161: alpha = 0.63024
Module 170: alpha = 0.49241
Module 178: alpha = 0.55618
Module 190: alpha = 0.28791
Module 193: alpha = 0.20577
Module 196: alpha = 0.34167
Module 204: alpha = 0.25783
